In [1]:
import torch
import torchvision
import pytorch_lightning as pl
from matplotlib import pyplot as plt

from Models import U_net, PlottingCallback
from Simulation import DataGenerator
import warnings
warnings.filterwarnings("ignore") 

c:\ProgramData\Anaconda3\envs\dl\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Dataset prep
DataGenerator.img_size = 200
dt = DataGenerator('train', size=100)
dv = DataGenerator('valid', size=10)
dt.generate()
dv.generate()

train_loader = torch.utils.data.DataLoader(dt, batch_size = 10, shuffle = True)
valid_loader = torch.utils.data.DataLoader(dv, batch_size = 10, shuffle = False)

# Model and Train
model = U_net(img_size=[dt.img_size, dt.img_size], learning_rate=1e-3)
trainer = pl.Trainer(gpus=-1, fast_dev_run = False, progress_bar_refresh_rate=20, callbacks=[PlottingCallback(dataloader=valid_loader)])
trainer.fit(model, train_loader, valid_loader)

100%|██████████| 10/10 [00:00<00:00, 263.75it/s]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type       | Params | In sizes                             | Out sizes                           
-----------------------------------------------------------------------------------------------------------------------------
0 | enc_1           | Encoder    | 828    | [1, 1, 200, 200]                     | [[1, 9, 100, 100], [1, 9, 100, 100]]
1 | enc_2           | Encoder    | 4.4 K  | [1, 9, 100, 100]                     | [[1, 18, 50, 50], [1, 18, 50, 50]]  
2 | enc_3           | Encoder    | 17.6 K | [1, 18, 50, 50]                      | [[1, 36, 25, 25], [1, 36, 25, 25]]  
3 | dec_3           | Decoder    | 8.8 K  | [[1, 36, 25, 25], [1, 36, 25, 25]]   | [1, 18, 50, 50]                     
4 | dec_2           | Decoder    | 2.

Epoch 146:   0%|          | 0/11 [00:00<?, ?it/s, loss=0.00637, v_num=10]         

Epoch 146:   0%|          | 0/11 [00:15<?, ?it/s, loss=0.00637, v_num=10]